<a href="https://colab.research.google.com/github/Suryansh109/project_covid_v1/blob/patch-1/IndianFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#setting up the spark and hadoop setup
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
#installing kaggle library to fetch the dataset  directly from kaggle and load it colab cluster
!pip install -q kaggle


In [22]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-3.0.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [23]:
import findspark
findspark.init()

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [6]:
#from google.colab.output import eval_js
#print(eval_js("google.colab.kernel.proxyPort(4040)"))

https://ejwrqhwb1eg-496ff2e9c6d22116-4040-colab.googleusercontent.com/


In [9]:
#to make kaggle.json directory 
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"suryansh1001","key":"4e01b75bf9b07a8f316ccec7388c6c6c"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d nehaprabhavalkar/indian-food-101

  0% 0.00/6.79k [00:00<?, ?B/s]
100% 6.79k/6.79k [00:00<00:00, 6.04MB/s]


In [25]:
!ls -ltr

total 437668
drwxr-xr-x 13 1000 1000      4096 Aug 28 09:22 spark-3.0.1-bin-hadoop3.2
-rw-r--r--  1 root root 224062525 Aug 28 09:25 spark-3.0.1-bin-hadoop3.2.tgz.1
-rw-r--r--  1 root root 224062525 Aug 28 09:25 spark-3.0.1-bin-hadoop3.2.tgz
-rw-r--r--  1 root root     28306 Sep 30 06:23 indian_food.csv
drwxr-xr-x  1 root root      4096 Dec  2 22:04 sample_data
-rw-r--r--  1 root root      6951 Dec 11 15:59 indian-food-101.zip


In [13]:
!unzip indian-food-101.zip

Archive:  indian-food-101.zip
  inflating: indian_food.csv         


In [26]:
raw_df=spark.read.option("header",True).option("inferSchema",True).csv("indian_food.csv")

In [59]:
from pyspark.sql import functions as f

In [73]:
raw_df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

quering the above dataset would be troublesome as the table is not normalized.



In [69]:
processed_df=raw_df.withColumn("ingredients",f.explode(f.split(col("ingredients"),",")))

In [72]:
processed_df.show()

+--------------+-----------------+----------+---------+---------+--------------+-------+-----------+------+
|          name|      ingredients|      diet|prep_time|cook_time|flavor_profile| course|      state|region|
+--------------+-----------------+----------+---------+---------+--------------+-------+-----------+------+
|    Balu shahi|      Maida flour|vegetarian|       45|       25|         sweet|dessert|West Bengal|  East|
|    Balu shahi|           yogurt|vegetarian|       45|       25|         sweet|dessert|West Bengal|  East|
|    Balu shahi|              oil|vegetarian|       45|       25|         sweet|dessert|West Bengal|  East|
|    Balu shahi|            sugar|vegetarian|       45|       25|         sweet|dessert|West Bengal|  East|
|        Boondi|       Gram flour|vegetarian|       80|       30|         sweet|dessert|  Rajasthan|  West|
|        Boondi|             ghee|vegetarian|       80|       30|         sweet|dessert|  Rajasthan|  West|
|        Boondi|            

The dish with highest number of ingredients?

In [76]:
highest_ingredients_food=processed_df.groupby("name").count().orderBy("count",ascending=False)

In [78]:
highest_ingredients_food.show()

+-----------------+-----+
|             name|count|
+-----------------+-----+
|           Ghevar|   10|
|           Jalebi|    9|
|      Gulab jamun|    8|
|       Mysore pak|    8|
|   Obbattu holige|    7|
|     Bandar laddu|    7|
|       Maach Jhol|    6|
|     Chana masala|    6|
|   Chicken razala|    6|
|Sabudana Khichadi|    6|
|       Aloo matar|    6|
|            Galho|    6|
|   Gajar ka halwa|    6|
|     Thayir sadam|    6|
|      Koldil Duck|    6|
|        Nankhatai|    6|
|         Vindaloo|    6|
|        Cham cham|    6|
|   Butter chicken|    6|
|         Beef Fry|    6|
+-----------------+-----+
only showing top 20 rows



Which ingredient has been used most in foods?

In [93]:
processed_df.groupBy("ingredients").agg(f.collect_list("name")).withColumn("count",col("collect_list(name)")).show()

+------------------+--------------------+--------------------+
|       ingredients|  collect_list(name)|               count|
+------------------+--------------------+--------------------+
|             beans|[Kootu, Poriyal, ...|[Kootu, Poriyal, ...|
|         coriander|[Paratha, Bengena...|[Paratha, Bengena...|
| frozen green peas|    [Mushroom matar]|    [Mushroom matar]|
| whole wheat flour|     [Fara, Muthiya]|     [Fara, Muthiya]|
|     Banana flower|    [Koldil Chicken]|    [Koldil Chicken]|
|       alum powder|             [Petha]|             [Petha]|
|           mustard|[Poriyal, Zunka, ...|[Poriyal, Zunka, ...|
|         Moong dal|[Daal baati churm...|[Daal baati churm...|
|      dried fruits|             [Kheer]|             [Kheer]|
|             Palak|[Makki di roti sa...|[Makki di roti sa...|
|  musk melon seeds|          [Panjeeri]|          [Panjeeri]|
| whole wheat bread|     [Chicken Tikka]|     [Chicken Tikka]|
|         red chili|    [Keerai poriyal]|    [Keerai po